# ROS: Turtlesim
---

### Requirements:
- `ros-noetic-turtlesim`

In [ ]:
import jupyros
import rospy
import math
from turtlesim.srv import Spawn
from turtlesim.msg import Pose
from time import time
from jupyros.ros1 import TurtleSim

rospy.init_node("superturtle")

In [ ]:
turtlesim = TurtleSim(background_color="#0000FF")
display(turtlesim.canvas)

**TIP:** When using JupyterLab, you can right-click on the canvas and select *Create New View from Output*

In [ ]:
# Canvas default size is 1600x1600 starting from top-left corner
turtlesim.spawn(name="simon", pose={"x": 600, "y": 350, "theta": 20})

turtlesim.spawn(pose={"x": 1050, "y": 1250, "theta": 90})

# Change the turtle path color
turtlesim.turtles["turtle1"].path_color = "#F2F2F2"

Open two terminals and run:

```sh
rosrun turtlesim turtlesim_node
rosrun turtlesim turtle_teleop_key
```

### Subscriber

In [ ]:
poses = {}

for name in turtlesim.turtles.keys():
    poses[name] = turtlesim.turtles[name].pose
    
print(poses["turtle1"])

In [ ]:
topic_name = '/turtle1/pose'

# Based on the dimensions of window controlled by teleop keyboard
def convert_xy_units(x, y):
    convert_factor = 11.08
    x_convert = x / convert_factor * turtlesim.canvas.width
    y_convert = (convert_factor - y) / convert_factor * turtlesim.canvas.height
    
    return x_convert, y_convert

def callback_move_turtles(msg):
    name = "turtle1"
    x, y = convert_xy_units(msg.x, msg.y)
    poses[name] = {"x": x,
                   "y": y,
                   "theta": msg.theta}
    
    turtlesim.move_turtles(new_poses=poses)

jupyros.subscribe(topic_name, Pose, lambda msg: callback_move_turtles(msg))

### Spiral Publisher

In [ ]:
run = True

In [ ]:
%%thread_cell

rate = rospy.Rate(25)
pub = rospy.Publisher(topic_name, Pose, queue_size=10, latch=True)
i = 0

while run:
    msg = Pose()
    msg.x = (i/1800*5)*math.sin(i / 180 * math.pi) + 11.08 / 2
    msg.y = (i/1800*5)*math.cos(i / 180 * math.pi) + 11.08 / 2
    msg.theta = - i / 180 * math.pi
    pub.publish(msg)
    rate.sleep()
    i += 1
print("Done")

In [ ]:
run = False

## Spawn Service and Client

In [ ]:
def spawn_turtle(msg):
    print(f"Spawning turtle at x={msg.x} and y={msg.y}")
    turtlesim.spawn(name=msg.name, 
                    pose={"x": msg.x, 
                          "y": msg.y, 
                          "theta": msg.theta})
    return "Spawned"

In [ ]:
%%thread_cell

srv = rospy.Service('spawn_srv', Spawn, spawn_turtle)

In [ ]:
jupyros.client('spawn_srv', Spawn)